<a href="https://colab.research.google.com/github/ifeLight/ml-bot/blob/main/binance-grade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas-ta
!pip install backtrader
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=a8a4b55e98c16a1b935158eb8732b45cd4b79c8e67ff2972716391d8e13aeca5
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import requests
import json
import datetime
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import backtrader as bt
import pandas_ta as ta
from imblearn.over_sampling import SMOTE

In [3]:

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects

binance_base_url = 'https://52on3577u3.execute-api.eu-central-1.amazonaws.com'

def get_binance_candles(symbol: str, interval='1h', limit: int = 50, **kwargs):
    url = f'{binance_base_url}/api/v3/uiKlines?symbol={symbol}&interval={interval}&limit={limit}'
    for key, value in kwargs.items():
        url += f'&{key}={value}'
    response = requests.get(url)
    result = json.loads(response.text)
    # print(result)
    def map_result(x):
        return {
            'Date': x[0],
            'Open': x[1],
            'High': x[2],
            'Low': x[3],
            'Close': x[4],
            'Volume': x[5],
        }
    mappeded_result = []
    for x in result:
        mappeded_result.append(map_result(x))
    return mappeded_result


def candles_to_df(data):
    df =  pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'], unit='ms')
    df['Open'] = df['Open'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Close'] = df['Close'].astype(float)
    df['Volume'] = df['Volume'].astype(float)
    df.set_index('Date', inplace=True)
    return df

def get_all_binance_candles(symbol: str, interval='1h', start_date=None, end_date=None, limit=1000):
  result = []
  start_date = pd.to_datetime(start_date) if start_date else pd.to_datetime('2015-01-01')
  end_date = pd.to_datetime(end_date) if end_date else pd.to_datetime('today')
  while True:
    candles = get_binance_candles(symbol, interval, limit, startTime=int(start_date.timestamp() * 1000), endTime=int(end_date.timestamp() * 1000))
    if len(candles) <= 1:
      break;
    result += candles
    start_date = pd.to_datetime(datetime.datetime.fromtimestamp(candles[-1]['Date'] / 1000))
  return candles_to_df(result)

In [4]:
data = get_all_binance_candles('BTCUSDT', '4h', '01-01-2023', '01-02-2025')
data

,Open,High,Low,Close,Volume
Date,,,,,
2023-01-01 00:00:00,16541.77,16559.77,16508.39,16533.04,15515.82327
2023-01-01 04:00:00,16533.04,16550.00,16499.01,16526.19,16532.24115
2023-01-01 08:00:00,16525.70,16557.00,16505.20,16556.66,15915.96701
2023-01-01 12:00:00,16556.66,16572.94,16533.68,16558.73,15046.09096
2023-01-01 16:00:00,16558.73,16623.65,16558.00,16603.08,18532.64857
...,...,...,...,...,...
2025-01-01 08:00:00,93684.11,93756.02,92888.00,93444.64,1867.48363
2025-01-01 12:00:00,93444.65,94500.00,93407.07,94432.94,2077.71092
2025-01-01 16:00:00,94432.94,94823.75,93760.93,94627.07,1721.05382


In [5]:
def add_pivots(df, window=5):
    """
    Calculate the probability of price going up and down based on higher and lower pivots.
    """
    df['Higher_Pivot'] = df['High'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.max() else 0, raw=False
    )
    df['Lower_Pivot'] = df['Low'].rolling(window=2 * window + 1, center=True).apply(
        lambda x: 1 if x.iloc[window] == x.min() else 0, raw=False
    )

    # Step 2: Ensure no two successive pivots of the same type
    pivot_type = None  # Tracks the type of the last pivot
    last_pivot_index = None  # Tracks the index of the last pivot

    # Remove duplicated index
    df = df[~df.index.duplicated(keep='first')]

    for i in range(len(df.index)):
      if df.loc[df.index[i], 'Higher_Pivot'].item() == 1:
          if pivot_type == 'higher':
              # Compare current higher pivot with the last higher pivot
              if df.loc[df.index[i], 'High'].item() > df.loc[df.index[last_pivot_index], 'High'].item():
                  # Remove the last higher pivot
                  df.loc[df.index[last_pivot_index], 'Higher_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current higher pivot
                  df.loc[df.index[i], 'Higher_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'higher'
              last_pivot_index = i

      elif df.loc[df.index[i], 'Lower_Pivot'].item() == 1:
          if pivot_type == 'lower':
              # Compare current lower pivot with the last lower pivot
              if df.loc[df.index[i], 'Low'].item() < df.loc[df.index[last_pivot_index], 'Low'].item():
                  # Remove the last lower pivot
                  df.loc[df.index[last_pivot_index], 'Lower_Pivot'] = 0
                  # Update the last pivot
                  last_pivot_index = i
              else:
                  # Remove the current lower pivot
                  df.loc[df.index[i], 'Lower_Pivot'] = 0
          else:
              # Update pivot type and index
              pivot_type = 'lower'
              last_pivot_index = i
    return df

In [6]:
data =add_pivots(data, 10)

In [7]:
def plot_pivots(data):
  candlestick = go.Candlestick(
      x=data.index,
      open=data['Open'],
      high=data['High'],
      low=data['Low'],
      close=data['Close'],
      name='Candlestick'
  )

  # Step 3: Add markers for higher and lower pivots
  higher_pivots_df = data[data['Higher_Pivot'] == 1]
  lower_pivots_df = data[data['Lower_Pivot'] == 1]

  higher_pivots = go.Scatter(
      x=higher_pivots_df.index,
      y=higher_pivots_df['High'],
      mode='markers',
      marker=dict(color='green', size=10, symbol='triangle-down'),
      name='Higher Pivot'
  )

  lower_pivots = go.Scatter(
      x=lower_pivots_df.index,
      y=lower_pivots_df['Low'],
      mode='markers',
      marker=dict(color='red', size=10, symbol='triangle-up'),
      name='Lower Pivot'
  )

  # Step 4: Create the figure
  fig = go.Figure(data=[candlestick, higher_pivots, lower_pivots])

  # Update layout
  fig.update_layout(
      title='Candlestick Chart with Higher and Lower Pivots',
      xaxis_title='Date',
      yaxis_title='Price',
      xaxis_rangeslider_visible=False,
      template='plotly_dark'
  )

  # Show the plot
  fig.show()

In [8]:
plot_pivots(data)

In [9]:
data['Pivot'] = np.where(data['Higher_Pivot'] == 1, -1, np.where(data['Lower_Pivot'] == 1, 1, 0))
del data['Higher_Pivot']
del data['Lower_Pivot']
data

<ipython-input-9-4b01a0a1c0f2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot
Date,,,,,,
2023-01-01 00:00:00,16541.77,16559.77,16508.39,16533.04,15515.82327,0
2023-01-01 04:00:00,16533.04,16550.00,16499.01,16526.19,16532.24115,0
2023-01-01 08:00:00,16525.70,16557.00,16505.20,16556.66,15915.96701,0
2023-01-01 12:00:00,16556.66,16572.94,16533.68,16558.73,15046.09096,0
2023-01-01 16:00:00,16558.73,16623.65,16558.00,16603.08,18532.64857,0
...,...,...,...,...,...,...
2025-01-01 08:00:00,93684.11,93756.02,92888.00,93444.64,1867.48363,0
2025-01-01 12:00:00,93444.65,94500.00,93407.07,94432.94,2077.71092,0
2025-01-01 16:00:00,94432.94,94823.75,93760.93,94627.07,1721.05382,0


In [61]:
def calculate_pivot_proximity(df):
    # Initialize the new column with zeros
    df['Pivot_Proximity'] = 0.0

    # Iterate over the DataFrame to calculate proximity
    for i, row in df.iterrows():
      if row['Pivot'] == -1:
        # Higher pivot, set to -1
        df.at[i, 'Pivot_Proximity'] = -1.0
      elif row['Pivot'] == 1:
        # Lower pivot, set to 1
        df.at[i, 'Pivot_Proximity'] = 1.0
      else:
        # Not a pivot, calculate proximity
        # Find the closest previous pivot
        previous_pivots = df[(df['Pivot'] != 0) & (df.index < i)]

        if not previous_pivots.empty:
          closest_previous_pivot = previous_pivots.iloc[-1]  # Most recent previous pivot

          if closest_previous_pivot['Pivot'] == -1:
            # Previous pivot is a higher pivot, find the next lower pivot
            next_pivot = df[(df['Pivot'] == 1) & (df.index > i)]

            if not next_pivot.empty:
              closest_next_pivot = next_pivot.iloc[0]  # Closest next lower pivot

              # Calculate distances
              distance_to_previous = abs(row['Close'] - closest_previous_pivot['Close'])
              distance_to_next = abs(row['Close'] - closest_next_pivot['Close'])

              # Calculate the proximity value
              if distance_to_previous + distance_to_next != 0:
                proximity = (distance_to_previous - distance_to_next) / (distance_to_previous + distance_to_next)
                df.at[i, 'Pivot_Proximity'] = proximity
              else:
                # If both distances are zero, set proximity to 0
                df.at[i, 'Pivot_Proximity'] = 0.0
            else:
                # If no next lower pivot is found, set proximity to 0
                  df.at[i, 'Pivot_Proximity'] = 0.0
          elif closest_previous_pivot['Pivot'] == 1:
              # Previous pivot is a lower pivot, find the next higher pivot
              next_pivot = df[(df['Pivot'] == -1) & (df.index > i)]

              if not next_pivot.empty:
                closest_next_pivot = next_pivot.iloc[0]  # Closest next higher pivot

                # Calculate distances
                distance_to_previous = abs(row['Close'] - closest_previous_pivot['Close'])
                distance_to_next = abs(row['Close'] - closest_next_pivot['Close'])

                # Calculate the proximity value
                if distance_to_previous + distance_to_next != 0:
                  proximity = (distance_to_next - distance_to_previous) / (distance_to_previous + distance_to_next)
                  df.at[i, 'Pivot_Proximity'] = proximity
                else:
                  # If both distances are zero, set proximity to 0
                  df.at[i, 'Pivot_Proximity'] = 0.0
              else:
                  # If no next higher pivot is found, set proximity to 0
                  df.at[i, 'Pivot_Proximity'] = 0.0
          else:
            # If no previous pivot is found, set proximity to 0
            df.at[i, 'Pivot_Proximity'] = 0.0

    return df

In [62]:
data = calculate_pivot_proximity(data)
data

<ipython-input-61-42a07bd586b0>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2023-01-09 04:00:00,17197.00,17262.56,17180.00,17198.48,33546.34885,0,0.0,0.744564,0.802187,0.995065,0.986609,0.977939,0.005173,0.001167,0.004006,0.972509,0.985611,0.998714,2.658804,1.049074
2023-01-09 08:00:00,17198.01,17285.48,17186.95,17238.93,38468.19722,0,0.0,0.762591,0.828283,0.995153,0.985722,0.976464,0.005750,0.001403,0.004347,0.969919,0.984474,0.999029,2.956925,1.033342
2023-01-09 12:00:00,17239.47,17335.35,17190.74,17268.49,50514.03168,0,0.0,0.775083,0.846088,0.995633,0.985486,0.975652,0.006272,0.001546,0.004726,0.968614,0.984163,0.999711,3.159739,1.009284
2023-01-09 16:00:00,17268.49,17398.80,17268.02,17293.69,52520.91466,0,0.0,0.785448,0.860849,0.996122,0.985500,0.975082,0.006724,0.001604,0.005120,0.968641,0.984335,1.000029,3.188733,0.999089
2023-01-09 20:00:00,17294.19,17313.30,17128.00,17178.26,40362.87068,0,0.0,0.639969,0.563697,1.001877,0.992838,0.982356,0.006520,0.001092,0.005428,0.976449,0.991931,1.007414,3.121673,0.760581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 08:00:00,93684.11,93756.02,92888.00,93444.64,1867.48363,0,0.0,0.455667,0.446403,1.002746,1.006215,1.020262,-0.004295,0.001060,-0.005355,0.986169,1.004128,1.022088,3.577108,0.385063
2025-01-01 12:00:00,93444.65,94500.00,93407.07,94432.94,2077.71092,0,0.0,0.510426,0.580650,0.994834,0.996076,1.009626,-0.003394,0.001524,-0.004918,0.976545,0.993175,1.009806,3.348904,0.705191
2025-01-01 16:00:00,94432.94,94823.75,93760.93,94627.07,1721.05382,0,0.0,0.520628,0.603324,0.995196,0.994575,1.007603,-0.002515,0.001915,-0.004429,0.974911,0.990923,1.006935,3.231734,0.783429


In [96]:
def plot_candlestick_with_proximity(df, col='Pivot_Proximity', limit=0.0):
    # Create a color list based on Pivot_Proximity
    colors = []
    for proximity in df[col]:
        if proximity > (0 + limit):
            colors.append('green')  # Positive proximity (closer to lower pivot)
        elif proximity < (0 + (-1 * limit)):
            colors.append('red')    # Negative proximity (closer to higher pivot)
        else:
            colors.append('gray')   # Neutral (middle or no pivot)

    # Create the candlestick trace
    candlestick = go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Candlesticks',
        increasing=dict(line=dict(color='green'), fillcolor='green'),
        decreasing=dict(line=dict(color='red'), fillcolor='red'),
    )

    # Create a bar trace for Pivot_Proximity coloring
    proximity_trace = go.Bar(
        x=df.index,
        y=df['Close'],
        marker=dict(color=colors),
        opacity=0.3,
        name='Pivot Proximity',
        yaxis='y2'
    )

    # Create pivot indicators
    higher_pivots_df = data[data[col] == -1]
    lower_pivots_df = data[data[col] == 1]

    higher_pivots = go.Scatter(
        x=higher_pivots_df.index,
        y=higher_pivots_df['High'],
        mode='markers',
        marker=dict(color='red', size=10, symbol='triangle-down'),
        name='Higher Pivot'
    )

    lower_pivots = go.Scatter(
        x=lower_pivots_df.index,
        y=lower_pivots_df['Low'],
        mode='markers',
        marker=dict(color='green', size=10, symbol='triangle-up'),
        name='Lower Pivot'
    )

    # Create the figure
    fig = go.Figure(data=[candlestick, higher_pivots, lower_pivots, proximity_trace])

    # Update layout
    fig.update_layout(
        title='Candlestick Chart with Pivot Proximity',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2=dict(title='Pivot Proximity', overlaying='y', side='right'),
        template='plotly_dark',
        showlegend=True,
        xaxis_rangeslider_visible=False,
    )

    # Show the figure
    fig.show()

In [89]:
plot_candlestick_with_proximity(data, 'Pivot_Proximity', 0.5)

In [66]:
features_columns = []

def add_scaled_rsi(df, window=14):
  df.ta.rsi(length=window, append=True)
  df[f"RSI_{window}"] = df[f"RSI_{window}"] / 100
  features_columns.append(f"RSI_{window}")
  return df

def add_scaled_ema(df, window=50):
  df.ta.ema(length=window, append=True)
  df[f"EMA_{window}"] = df[f"EMA_{window}"] / df['Close']
  features_columns.append(f"EMA_{window}")
  return df

def add_scaled_sma(df, window=50):
  df.ta.sma(length=window, append=True)
  df[f"SMA_{window}"] = df[f"SMA_{window}"] / df['Close']
  features_columns.append(f"SMA_{window}")
  return df

def add_scaled_macd(df):
  df.ta.macd(append=True)
  df[f"MACD_12_26_9"] = df[f"MACD_12_26_9"] / df['Close']
  df[f"MACDs_12_26_9"] = df[f"MACDs_12_26_9"] / df['Close']
  df[f"MACDh_12_26_9"] = df[f"MACDh_12_26_9"] / df['Close']
  features_columns.append(f"MACD_12_26_9")
  features_columns.append(f"MACDs_12_26_9")
  features_columns.append(f"MACDh_12_26_9")
  return df

def add_scaled_bbands(df, window=20, std=2.0):
  df.ta.bbands(length=window, append=True, std=std)
  df[f"BBL_{window}_{std}"] = df[f"BBL_{window}_{std}"] / df['Close']
  df[f"BBM_{window}_{std}"] = df[f"BBM_{window}_{std}"] / df['Close']
  df[f"BBU_{window}_{std}"] = df[f"BBU_{window}_{std}"] / df['Close']
  features_columns.append(f"BBL_{window}_{std}")
  features_columns.append(f"BBM_{window}_{std}")
  features_columns.append(f"BBU_{window}_{std}")
  features_columns.append(f"BBB_{window}_{std}")
  features_columns.append(f"BBP_{window}_{std}")
  return df

In [67]:
# # Add technical indicators
add_scaled_rsi(data, 14)
add_scaled_rsi(data, 6)
add_scaled_ema(data, 5)
add_scaled_ema(data, 21)
add_scaled_sma(data, 50)
add_scaled_macd(data)
add_scaled_bbands(data)
data

<ipython-input-66-033a3a108830>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-66-033a3a108830>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-66-033a3a108830>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-66-033a3a1

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2023-01-09 04:00:00,17197.00,17262.56,17180.00,17198.48,33546.34885,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09 08:00:00,17198.01,17285.48,17186.95,17238.93,38468.19722,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09 12:00:00,17239.47,17335.35,17190.74,17268.49,50514.03168,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09 16:00:00,17268.49,17398.80,17268.02,17293.69,52520.91466,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-09 20:00:00,17294.19,17313.30,17128.00,17178.26,40362.87068,0,0.0,NaN,NaN,1.003336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 08:00:00,93684.11,93756.02,92888.00,93444.64,1867.48363,0,0.0,0.455667,0.446403,1.002746,1.006215,1.020262,-0.004295,0.001060,-0.005355,0.986169,1.004128,1.022088,3.577108,0.385063
2025-01-01 12:00:00,93444.65,94500.00,93407.07,94432.94,2077.71092,0,0.0,0.510426,0.580650,0.994834,0.996076,1.009626,-0.003394,0.001524,-0.004918,0.976545,0.993175,1.009806,3.348904,0.705191
2025-01-01 16:00:00,94432.94,94823.75,93760.93,94627.07,1721.05382,0,0.0,0.520628,0.603324,0.995196,0.994575,1.007603,-0.002515,0.001915,-0.004429,0.974911,0.990923,1.006935,3.231734,0.783429


In [68]:
features_columns

['RSI_14',
 'RSI_6',
 'EMA_5',
 'EMA_21',
 'SMA_50',
 'MACD_12_26_9',
 'MACDs_12_26_9',
 'MACDh_12_26_9',
 'BBL_20_2.0',
 'BBM_20_2.0',
 'BBU_20_2.0',
 'BBB_20_2.0',
 'BBP_20_2.0']

In [69]:
# Drop rows with NaN values (due to rolling calculations)
data.dropna(inplace=True)
data

<ipython-input-69-01bd4bc5b426>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2023-01-17 08:00:00,21133.65,21249.46,21082.48,21219.75,32766.90672,0,0.0,0.715645,0.654145,0.996866,0.974111,0.905413,0.026758,-0.004629,0.031387,0.970075,0.987147,1.004219,3.458812,0.876444
2023-01-17 12:00:00,21219.21,21647.45,21030.00,21176.06,87104.51182,0,0.0,0.698923,0.609604,0.999282,0.978291,0.911038,0.025462,-0.004792,0.030254,0.972144,0.989842,1.007540,3.575913,0.786994
2023-01-17 16:00:00,21176.06,21412.00,21084.00,21336.46,48658.53141,0,0.0,0.724386,0.699689,0.994513,0.973579,0.908030,0.024532,-0.004395,0.028928,0.965482,0.983841,1.002199,3.732006,0.940099
2023-01-17 20:00:00,21336.46,21413.15,21121.71,21134.81,35702.53057,0,0.0,0.649966,0.519013,1.002668,0.984425,0.920352,0.023139,-0.004852,0.027991,0.975854,0.994051,1.012249,3.661290,0.663448
2023-01-18 00:00:00,21132.29,21377.44,21103.15,21258.54,40301.19947,0,0.0,0.672218,0.595854,0.997888,0.980632,0.918726,0.021939,-0.004711,0.026650,0.971289,0.989383,1.007476,3.657517,0.793407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-01 08:00:00,93684.11,93756.02,92888.00,93444.64,1867.48363,0,0.0,0.455667,0.446403,1.002746,1.006215,1.020262,-0.004295,0.001060,-0.005355,0.986169,1.004128,1.022088,3.577108,0.385063
2025-01-01 12:00:00,93444.65,94500.00,93407.07,94432.94,2077.71092,0,0.0,0.510426,0.580650,0.994834,0.996076,1.009626,-0.003394,0.001524,-0.004918,0.976545,0.993175,1.009806,3.348904,0.705191
2025-01-01 16:00:00,94432.94,94823.75,93760.93,94627.07,1721.05382,0,0.0,0.520628,0.603324,0.995196,0.994575,1.007603,-0.002515,0.001915,-0.004429,0.974911,0.990923,1.006935,3.231734,0.783429


In [70]:
# delete Pivot_Porximity that have zero at the beginning and ending of the dataframe
non_zero_indices = data[data['Pivot_Proximity'] != 0].index
non_zero_at_begining = non_zero_indices[0]
non_zero_at_end = non_zero_indices[-1]
data = data.loc[non_zero_at_begining:non_zero_at_end]
data

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2023-01-18 12:00:00,21204.67,21650.00,20842.00,21042.73,97538.81068,-1,-1.000000,0.586485,0.412547,1.005984,0.994588,0.939523,0.018015,-0.005279,0.023294,0.985974,1.002252,1.018529,3.248258,0.430838
2023-01-18 16:00:00,21042.73,21136.80,20407.15,20906.54,112546.34199,1,1.000000,0.543593,0.337126,1.008358,1.000970,0.949138,0.015911,-0.006027,0.021939,0.994350,1.009221,1.024092,2.947031,0.189958
2023-01-18 20:00:00,20905.37,20938.67,20624.02,20677.47,40606.82944,0,0.837660,0.480005,0.246257,1.013019,1.010963,0.962893,0.013261,-0.007136,0.020398,1.002785,1.019844,1.036902,3.345369,-0.081624
2023-01-19 00:00:00,20677.47,20817.85,20659.19,20748.04,37267.51378,0,0.881759,0.499432,0.314558,1.006382,1.006840,0.962814,0.011130,-0.007358,0.018489,0.998184,1.016081,1.033977,3.522739,0.050746
2023-01-19 04:00:00,20748.04,20874.81,20742.71,20818.95,28848.62637,0,0.931008,0.518882,0.382074,1.001970,1.003101,0.962780,0.009609,-0.007054,0.016662,0.994341,1.012494,1.030646,3.585769,0.155880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-29 20:00:00,93612.89,93949.52,93009.52,93738.20,3936.76599,0,0.531537,0.391479,0.293035,1.005167,1.014598,1.023193,-0.006838,-0.000691,-0.006147,0.995994,1.013374,1.030754,3.430161,0.115244
2024-12-30 00:00:00,93738.19,94274.29,93368.93,93722.71,2898.63024,0,0.536050,0.390668,0.290925,1.003555,1.013423,1.022605,-0.007144,-0.000797,-0.006347,0.994827,1.012272,1.029717,3.446703,0.148257
2024-12-30 04:00:00,93722.72,94060.35,93240.03,93731.65,2439.98241,0,0.533445,0.391452,0.294444,1.002307,1.012115,1.021748,-0.007293,-0.000757,-0.006536,0.993474,1.011077,1.028680,3.482093,0.185375


In [71]:
data[features_columns].describe()

,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDs_12_26_9,MACDh_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
count,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000,4273.000000
mean,0.521512,0.519732,0.999409,0.997004,0.992554,0.002140,0.002164,-0.000024,0.966549,0.997180,1.027811,6.169028,0.525897
std,0.131879,0.184222,0.009001,0.022183,0.041157,0.011603,0.010940,0.003403,0.034946,0.025173,0.029467,4.164642,0.340071
min,0.053691,0.007924,0.941419,0.880966,0.839649,-0.054377,-0.044037,-0.016898,0.710382,0.856471,0.959028,0.323249,-0.497445
25%,0.430976,0.393494,0.995832,0.985403,0.968821,-0.004538,-0.004242,-0.001904,0.953688,0.985112,1.009764,3.250591,0.287186
50%,0.516892,0.517751,0.999681,0.998404,0.996711,0.001188,0.001284,-0.000098,0.976406,0.998639,1.020150,5.154220,0.540153
75%,0.610836,0.647527,1.003216,1.008565,1.016365,0.008521,0.008525,0.001761,0.989606,1.009213,1.038022,7.871072,0.769816
max,0.916255,0.983481,1.067220,1.155367,1.243662,0.044217,0.038254,0.019638,1.042589,1.168764,1.304693,34.357976,1.574330


In [72]:
# Step 3: Prepare data for LSTM
def create_sequences(data, features_columns, target_col, seq_length):
    """
    Create sequences of `seq_length` time steps for LSTM input.
    """
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i + seq_length][features_columns].values)
        y.append(data.iloc[i + seq_length][target_col])
    return np.array(X), np.array(y)

In [73]:
def calc_class_ratios(df, column):
  class_counts = df[column].value_counts()
  total_count = len(df)
  class_ratios = class_counts / total_count
  return class_ratios.to_dict()

def class_ratio_to_class_weights(class_ratios):
  class_weights = {}
  for key, value in class_ratios.items():
    class_weights[key] = 1 / value
  return class_weights

class_ratios = calc_class_ratios(data, 'Pivot')
class_weights = class_ratio_to_class_weights(class_ratios)
class_weights

{0: 1.0587375091396538, 1: 35.90082644628099, -1: 36.2}

In [74]:
# select only first 80 % of the data
training_data = data[:int(len(data) * 0.8)]
training_data.tail(10)

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,EMA_21,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,,,
2024-08-08 12:00:00,57320.51,59505.00,56700.00,59428.03,14315.42395,0,-0.498888,0.587085,0.767964,0.970008,0.959470,1.001979,-0.007015,0.010133,-0.017148,0.883071,0.939746,0.996421,12.061697,1.031577
2024-08-08 16:00:00,59428.03,59863.00,59237.59,59414.51,6749.57668,0,-0.495888,0.586407,0.765726,0.980153,0.963353,0.999958,-0.003125,0.011221,-0.014346,0.895777,0.946755,0.997733,10.769014,1.022235
2024-08-08 20:00:00,59414.52,62745.14,59297.34,61685.99,11276.89671,-1,-1.000000,0.657860,0.852422,0.962707,0.934435,0.961615,0.002898,0.013373,-0.010475,0.857653,0.917862,0.978072,13.119508,1.182102
2024-08-09 00:00:00,61686.00,61744.37,60720.00,61358.39,5373.57013,0,-0.782425,0.640669,0.801114,0.978565,0.944931,0.965478,0.007108,0.014111,-0.007003,0.864654,0.929191,0.993728,13.891001,1.048595
2024-08-09 04:00:00,61358.39,61390.70,60588.63,60863.57,4475.52913,0,-0.453792,0.614547,0.722312,0.991014,0.956921,0.972091,0.009748,0.013447,-0.003698,0.875866,0.942368,1.008870,14.113848,0.933309
2024-08-09 08:00:00,60863.58,61210.31,60170.16,60655.24,4238.45087,0,-0.315430,0.603392,0.688115,0.996278,0.963826,0.974348,0.011427,0.012110,-0.000683,0.880279,0.949718,1.019157,14.623099,0.862060
2024-08-09 12:00:00,60655.25,61115.11,59535.00,59810.23,8756.37366,0,0.245781,0.559065,0.559243,1.006903,0.979493,0.986603,0.011636,0.009863,0.001773,0.895285,0.966321,1.037357,14.702375,0.737057
2024-08-09 16:00:00,59810.24,60750.00,59700.00,60681.42,3794.60746,0,-0.332818,0.592318,0.642156,0.994964,0.968574,0.971122,0.012521,0.008619,0.003902,0.886884,0.956956,1.027027,14.644693,0.807144
2024-08-09 20:00:00,60681.79,60986.03,59909.16,60837.99,4333.94892,0,-0.436803,0.598183,0.656107,0.994936,0.969165,0.967943,0.013373,0.007585,0.005788,0.886747,0.958136,1.029524,14.901634,0.793214


In [75]:
seq_length = 15
target_col = 'Pivot_Proximity'

# Create sequences
X, y = create_sequences(training_data, features_columns, target_col, seq_length)

In [78]:
print(X.shape)
print(y.shape)

(3403, 15, 13)
(3403,)


In [79]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [80]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (2722, 15, 13)
y_train shape: (2722,)
X_test shape: (681, 15, 13)
y_test shape: (681,)


In [81]:
# since the class weights index are now +1 to categorise dat
# adding one to the key of class weights
class_weights = {k+1: v for k, v in class_weights.items()}
class_weights

{1: 1.0587375091396538, 2: 35.90082644628099, 0: 36.2}

In [82]:
# Step 4: Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1, activation='tanh')
])
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.Accuracy, tf.keras.metrics.MeanSquaredError])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [83]:
# Step 5: Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1,
    # class_weight=class_weights
)

Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.0000e+00 - loss: 0.4286 - mean_squared_error: 0.4286 - val_accuracy: 0.0000e+00 - val_loss: 0.3651 - val_mean_squared_error: 0.3651
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.0000e+00 - loss: 0.3993 - mean_squared_error: 0.3993 - val_accuracy: 0.0000e+00 - val_loss: 0.3199 - val_mean_squared_error: 0.3199
Epoch 3/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.0000e+00 - loss: 0.3799 - mean_squared_error: 0.3799 - val_accuracy: 0.0000e+00 - val_loss: 0.3415 - val_mean_squared_error: 0.3415
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.0000e+00 - loss: 0.3612 - mean_squared_error: 0.3612 - val_accuracy: 0.0000e+00 - val_loss: 0.3121 - val_mean_squared_error: 0.3121
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.0000e+00 - loss: 0.3534 - mean_squared_error: 0.3534 - val_accuracy: 0.0000e+00 - val_loss: 0.3465 - val_mean_squared_error: 0.3465
Epoch 6/10

In [86]:
# Step 6: Evaluate the model
loss, _, mse = model.evaluate(X_test, y_test, verbose=0)
model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test MSE: {mse:.4f}")

Test Loss: 0.4717
Test MSE: 0.4717


In [87]:
# # Step 7: Predict pivots
# data['Predicted_Pivot'] = np.nan
# data['Confidence'] = np.nan
# for i in range(seq_length, len(data)):
#     # input_data = scaler.transform(data.iloc[i - seq_length:i][features_columns]) # If using a scaler
#     input_data = data.iloc[i - seq_length:i][features_columns]
#     input_data = np.expand_dims(input_data, axis=0)
#     prediction = model.predict(input_data, verbose=0)
#     predicted_class = np.argmax(prediction) - 1  # Convert back to -1, 0, 1
#     data.loc[data.index[i], 'Predicted_Pivot'] = predicted_class
#     data.loc[data.index[i], 'Confidence'] = np.max(prediction)

# Optimised version

# Pre-allocate arrays for predictions and confidence scores
predicted_values = np.full(len(data), np.nan)

# Convert feature columns to a NumPy array for faster access
feature_array = data[features_columns].values

# Iterate over the data and make predictions
for i in range(seq_length, len(data)):
    # Extract input data (no need for .iloc, use NumPy slicing)
     # input_data = scaler.transform(feature_array[i - seq_length:i]) # If using a scaler
    input_data = feature_array[i - seq_length:i]
    input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

    # Make prediction
    prediction = model.predict(input_data, verbose=0)
    predicted_class = prediction

    # Store results in pre-allocated arrays
    predicted_values[i] = predicted_class

# Assign the results back to the DataFrame in one go
data['Predicted_Value'] = predicted_values

<ipython-input-87-bfb9f7e21380>:33: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

<ipython-input-87-bfb9f7e21380>:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [90]:
data[['Open', 'High', 'Low', 'Close', 'Pivot', 'Predicted_Value']]

,Open,High,Low,Close,Pivot,Predicted_Value
Date,,,,,,
2023-01-18 12:00:00,21204.67,21650.00,20842.00,21042.73,-1,NaN
2023-01-18 16:00:00,21042.73,21136.80,20407.15,20906.54,1,NaN
2023-01-18 20:00:00,20905.37,20938.67,20624.02,20677.47,0,NaN
2023-01-19 00:00:00,20677.47,20817.85,20659.19,20748.04,0,NaN
2023-01-19 04:00:00,20748.04,20874.81,20742.71,20818.95,0,NaN
...,...,...,...,...,...,...
2024-12-29 20:00:00,93612.89,93949.52,93009.52,93738.20,0,0.211486
2024-12-30 00:00:00,93738.19,94274.29,93368.93,93722.71,0,0.534491
2024-12-30 04:00:00,93722.72,94060.35,93240.03,93731.65,0,0.654218


In [93]:
test_data = data[int(len(data) * 0.8):]
test_data.head(10)

,Open,High,Low,Close,Volume,Pivot,Pivot_Proximity,RSI_14,RSI_6,EMA_5,...,SMA_50,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Predicted_Value
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-10 04:00:00,60437.85,60851.91,60242.00,60842.00,1152.48879,0,-0.439466,0.592290,0.633479,0.996244,...,0.965415,0.013778,0.005176,0.008602,0.893654,0.965051,1.036449,14.796563,0.744748,-0.546882
2024-08-10 08:00:00,60842.01,61116.65,60601.96,60920.88,1618.24524,0,-0.491854,0.595671,0.643060,0.996636,...,0.962954,0.014012,0.004337,0.009675,0.895601,0.967133,1.038665,14.792661,0.729734,-0.573595
2024-08-10 12:00:00,60920.88,60949.99,60400.00,60515.89,1529.47703,0,-0.222881,0.569559,0.553859,1.002204,...,0.968193,0.013610,0.003096,0.010514,0.905360,0.976576,1.047792,14.584804,0.664460,-0.565272
2024-08-10 16:00:00,60515.90,61196.63,60400.00,61016.45,1789.44845,0,-0.555327,0.593290,0.629985,0.995988,...,0.959056,0.013613,0.002548,0.011065,0.901903,0.971885,1.041867,14.401284,0.700870,-0.371228
2024-08-10 20:00:00,61016.44,61160.00,60757.39,60923.51,1039.47144,0,-0.493601,0.586821,0.606911,0.998338,...,0.959700,0.013447,0.001892,0.011555,0.910717,0.977331,1.043945,13.631839,0.670149,-0.528399
2024-08-11 00:00:00,60923.51,61382.73,60892.00,61085.01,1397.39336,0,-0.600861,0.595083,0.634802,0.997133,...,0.956656,0.013324,0.001440,0.011884,0.923286,0.979925,1.036564,11.559878,0.677217,-0.492334
2024-08-11 04:00:00,61085.01,61271.43,60881.36,61257.00,1213.74092,0,-0.715087,0.604161,0.665163,0.996222,...,0.953891,0.013290,0.001152,0.012139,0.937399,0.982172,1.026944,9.117037,0.699100,-0.585723
2024-08-11 08:00:00,61257.00,61858.00,60425.00,60669.15,3535.80151,0,-0.324668,0.558105,0.496027,1.003917,...,0.962730,0.012496,0.000192,0.012304,0.956012,0.994671,1.033331,7.773289,0.568920,-0.612453
2024-08-11 12:00:00,60669.14,60761.09,59941.00,60127.29,4006.71160,0,0.035206,0.518843,0.387139,1.008642,...,0.970895,0.011017,-0.001119,0.012135,0.975241,1.006075,1.036908,6.129432,0.401491,-0.456376


In [98]:
plot_candlestick_with_proximity(test_data, 'Predicted_Value', 0.7)